In [2]:
pip install pdfplumber pytesseract pymupdf opencv-python pandas


  Using cached pdfplumber-0.11.7-py3-none-any.whl.metadata (42 kB)
  Using cached pdfminer_six-20250506-py3-none-any.whl.metadata (4.2 kB)
  Using cached pypdfium2-4.30.1-py3-none-win_amd64.whl.metadata (48 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached pdfplumber-0.11.7-py3-none-any.whl (60 kB)
Using cached pdfminer_six-20250506-py3-none-any.whl (5.6 MB)
   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/18.7 MB 10.9 MB/s eta 0:00:02
   -- ------------------------------------- 1.2/18.7 MB 13.0 MB/s eta 0:00:02
   ---- ----------------------------------- 1.9/18.7 MB 13.5 MB/s eta 0:00:02
   ----- ------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pdfplumber
import fitz  # PyMuPDF
import pytesseract
import pandas as pd
import io
from PIL import Image
import cv2
import numpy as np

def extract_tables_from_pdf(pdf_path):
    all_tables = []

    # --- Extract text-based tables using pdfplumber ---
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                tables = page.extract_tables()
                for table in tables:
                    if table:
                        df = pd.DataFrame(table[1:], columns=table[0])
                        all_tables.append(('text', i + 1, df))
    except Exception as e:
        print("Error extracting text-based tables:", e)

    # --- Extract image-based tables using OCR (PyMuPDF + pytesseract) ---
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            pix = page.get_pixmap(dpi=300)
            img_bytes = pix.tobytes("png")

            img = Image.open(io.BytesIO(img_bytes)).convert("RGB")
            img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)

            # Apply OCR
            custom_config = r'--oem 3 --psm 6'
            ocr_result = pytesseract.image_to_string(img_cv, config=custom_config)

            # Try to parse OCR result as a table
            lines = [line.strip() for line in ocr_result.split('\n') if line.strip()]
            if len(lines) > 2 and any('\t' in line or '  ' in line for line in lines):
                rows = [line.split() for line in lines]
                if all(len(row) == len(rows[0]) for row in rows):
                    df = pd.DataFrame(rows[1:], columns=rows[0])
                    all_tables.append(('ocr', page_num + 1, df))
    except Exception as e:
        print("Error extracting image-based tables:", e)

    return all_tables


In [6]:
a=extract_tables_from_pdf(r"C:\Users\Akshaya V\OneDrive\Desktop\The Press Release_2Q25.pdf")

Error extracting image-based tables: tesseract is not installed or it's not in your PATH. See README file for more information.


In [9]:
print(a[2])

('text', 2,   Total revenue, net of interest expense $26.5 $27.4 $25.4
0            Provision for credit losses 1.6 1.5 1.5      
1                 Noninterest expense 17.2 17.8 16.3      
2                          Pretax income 7.7 8.1 7.6      
3       Pretax, pre-provision income1(F) 9.3 9.6 9.1      
4                     Income tax expense 0.6 0.7 0.7      
5                             Net income 7.1 7.4 6.9      
6       Diluted earnings per share $0.89 $0.90 $0.83      )
